In [1]:
# pdp package path
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

# imports
import random
from collections import defaultdict
from core import * 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
## Define problems with sizes 10, 15, 20
# Each with an id 1, 2, 3, ... where (id // 3) * 5 + 10 = number of requests

possible_items = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(600, 300),
    TwoDimensionalItem(600, 400),
    TwoDimensionalItem(300, 200),
]

# 2 small compartments and 2 large compartments
small_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600), 
    ]
)

# 3 small compartments and 2 large compartments
medium_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600), 
    ]
)

# 3 small compartments and 3 large compartments
large_vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),  
    ]
)

vehicles = [small_vehicle, medium_vehicle, large_vehicle]

group_size = 3
instance_sizes = [10, 15, 20]
problem_ids = list(range(group_size * len(instance_sizes)))

# create random problems with seed based on id!
problems = []
modelled_problems = []
for prob_id in problem_ids:
    instance_size_idx = prob_id // group_size
    instance_size = instance_sizes[instance_size_idx]
    nb_items = instance_size
    vehicle = vehicles[instance_size_idx]
    V = list(range(0, nb_items * 2 + 1))
    random.seed(prob_id)
    items = random.choices(possible_items, k=nb_items)
    random.seed(prob_id)
    distance_matrix = [
        [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V]
        for j in V
    ]
    
    problem = TwoDimensionalProblem(items, vehicle, distance_matrix, name=f"prob_{prob_id}")
    problems.append(problem)

    # Modellel Problem for time-limited gurobi
    modelled_problem = ModelledTwoDimensionalProblem(items, vehicle, distance_matrix=distance_matrix)
    modelled_problem.create_model()
    modelled_problem.apply_constraints()
    modelled_problem.set_model_objective()
    modelled_problems.append(modelled_problem)


Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-11


In [4]:
# Initial, quick solutions by gurobi
initial_solutions_dict = {}
for prob_id, problem in enumerate(modelled_problems):
    initial_solution_time_limit = (prob_id // 3) + 1 # 1 sec for 10, 2 for 15, 3 for 20  
    modelled_problem = modelled_problems[prob_id]
    modelled_problem.solve(time_limit=initial_solution_time_limit)
    initial_solutions_dict[prob_id] = modelled_problem.extract_solution()

Set parameter TimeLimit to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 4127 rows, 680 columns and 19367 nonzeros
Model fingerprint: 0x92cb04c1
Variable types: 0 continuous, 680 integer (575 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 570 rows and 154 columns
Presolve time: 0.06s
Presolved: 3557 rows, 526 columns, 13502 nonzeros
Variable types: 0 continuous, 526 integer (426 binary)
Found heuristic solution: objective 1028.0000000

Root relaxation: objective 1.603000e+02, 347 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  160.30000    0   27 1028.00000  160.30000  84.4% 

In [5]:
import time

In [71]:
def get_gap(exact_val, heuristic_val):
    zp = heuristic_val
    zd = exact_val

    return abs(zp - zd) / abs(zp)

In [9]:
# greedy solvers
greedy_solvers = [
    GreedySolver(problem)
    for problem in problems
]

greedy_solutions = []
greedy_stats = dict()
for prob_id, solver in enumerate(greedy_solvers):
    st_time = time.time()
    solution = solver.solve(nb_solutions=1)[0]
    delta = time.time() - st_time
    greedy_solutions.append(solution)

    greedy_stats[prob_id] = dict(solution=solution, delta=delta, objective=problems[prob_id].evaluate_solution(solution))


In [54]:
for prob_id in problem_ids:
    stat = greedy_stats[prob_id]
    print(f"Greedy Sol: id={prob_id}, obj={stat['objective']}, solving_time={'{:.2f}'.format(1e6*stat['delta'])}")

Greedy Sol: id=0, obj=408, solving_time=415.80
Greedy Sol: id=1, obj=364, solving_time=297.55
Greedy Sol: id=2, obj=533, solving_time=330.69
Greedy Sol: id=3, obj=485, solving_time=666.38
Greedy Sol: id=4, obj=517, solving_time=1707.32
Greedy Sol: id=5, obj=468, solving_time=694.04
Greedy Sol: id=6, obj=381, solving_time=900.03
Greedy Sol: id=7, obj=432, solving_time=946.52
Greedy Sol: id=8, obj=665, solving_time=1138.45


In [81]:
stat

{'solution': Solution(
 	order=0, 1, 16, 11, 18, 2, 22, 31, 7, 8, 10, 14, 36, 15, 9, 34, 4, 19, 29, 38, 20, 6, 35, 21, 3, 26, 30, 40, 28, 39, 13, 24, 17, 12, 32, 5, 33, 37, 25, 23, 27, 
 	step: 0 vertex: 0 	0, 0, 0, 0, 0, 0
 	step: 1 vertex: 1 	400, 0, 0, 0, 0, 0
 	step: 2 vertex: 16 	800, 0, 0, 0, 0, 0
 	step: 3 vertex: 11 	800, 600, 0, 0, 0, 0
 	step: 4 vertex: 18 	800, 600, 600, 0, 0, 0
 	step: 5 vertex: 2 	800, 800, 600, 0, 0, 0
 	step: 6 vertex: 22 	800, 600, 600, 0, 0, 0
 	step: 7 vertex: 31 	800, 0, 600, 0, 0, 0
 	step: 8 vertex: 7 	800, 200, 600, 0, 0, 0
 	step: 9 vertex: 8 	800, 600, 600, 0, 0, 0
 	step: 10 vertex: 10 	800, 600, 600, 300, 0, 0
 	step: 11 vertex: 14 	800, 800, 600, 300, 0, 0
 	step: 12 vertex: 36 	400, 800, 600, 300, 0, 0
 	step: 13 vertex: 15 	800, 800, 600, 300, 0, 0
 	step: 14 vertex: 9 	800, 800, 600, 700, 0, 0
 	step: 15 vertex: 34 	800, 600, 600, 700, 0, 0
 	step: 16 vertex: 4 	800, 600, 600, 700, 400, 0
 	step: 17 vertex: 19 	800, 600, 600, 700, 700, 0
 

In [73]:
exact_objs = [194, 164, 199, 215, 189, 201, 193]
exact_times = [23.1, 24.65, 137.52, 365.29, 2678.30, 2127.43, 19502.54]
for prob_id in [0, 1, 2, 3, 4, 5, 6]:
    greedy_obj = greedy_stats[prob_id]["objective"]
    greedy_time = greedy_stats[prob_id]["delta"]
    exact_obj = exact_objs[prob_id]
    exact_time = exact_times[prob_id]
    gap = get_gap(exact_val=exact_obj, heuristic_val=greedy_obj) * 100
    speedup = (exact_time / greedy_time)
    print(f"for {prob_id} gap: {'{:.2f}'.format(gap)}, speedup: {'{:.2e}'.format(speedup)}") 

for 0 gap: 52.45, speedup: 5.56e+04
for 1 gap: 54.95, speedup: 8.28e+04
for 2 gap: 62.66, speedup: 4.16e+05
for 3 gap: 55.67, speedup: 5.48e+05
for 4 gap: 63.44, speedup: 1.57e+06
for 5 gap: 57.05, speedup: 3.07e+06
for 6 gap: 49.34, speedup: 2.17e+07


In [48]:
lns_solvers = []
# time_limits = [10, 20, 30]
time_limits = [5, 30, 60]
for prob_id, problem in enumerate(problems):
    initial_solution = initial_solutions_dict[prob_id]
    time_limit = time_limits[prob_id // 3]
    solver = LNS(problem, initial_solution=initial_solution, time_limit=time_limit)
    lns_solvers.append(solver)

In [49]:
for solver in lns_solvers:
    solver.search()


Terminating LNS in 155 iterations
Total time: 5.02 Best objective: 255 found in 133 iterations in 4.34 s

Terminating LNS in 192 iterations
Total time: 5.01 Best objective: 213 found in 52 iterations in 1.36 s

Terminating LNS in 188 iterations
Total time: 5.01 Best objective: 258 found in 45 iterations in 1.26 s

Terminating LNS in 225 iterations
Total time: 30.06 Best objective: 303 found in 131 iterations in 17.73 s

Terminating LNS in 244 iterations
Total time: 30.10 Best objective: 260 found in 24 iterations in 3.72 s

Terminating LNS in 209 iterations
Total time: 30.03 Best objective: 269 found in 77 iterations in 11.79 s

Terminating LNS in 115 iterations
Total time: 60.12 Best objective: 349 found in 114 iterations in 59.59 s

Terminating LNS in 120 iterations
Total time: 60.00 Best objective: 251 found in 59 iterations in 30.02 s

Terminating LNS in 119 iterations
Total time: 60.17 Best objective: 399 found in 90 iterations in 46.43 s



In [77]:
lns_solvers_2 = []
time_limits = [5, 30, 100]
for prob_id, problem in enumerate(problems):
    if prob_id < 6:
        continue
    initial_solution = initial_solutions_dict[prob_id]
    time_limit = time_limits[prob_id // 3]
    solver = LNS(problem, initial_solution=initial_solution, time_limit=time_limit)
    lns_solvers_2.append(solver)

In [78]:
for solver in lns_solvers_2:
    solver.search()

Terminating LNS in 200 iterations
Total time: 100.09 Best objective: 259 found in 66 iterations in 34.86 s

Terminating LNS in 210 iterations
Total time: 100.40 Best objective: 298 found in 200 iterations in 95.90 s

Terminating LNS in 212 iterations
Total time: 100.31 Best objective: 361 found in 154 iterations in 73.56 s



In [51]:
def get_best_before(max_time, solver_solutions):
    best_before = solver_solutions[1]
    for sol in solver_solutions:
        if sol.time_to_find <= max_time:
            if not best_before or sol.objective < best_before.objective:
                best_before = sol
        else:
            break
    return best_before


time_cutoff = 10
solutions_cutoff_dict = {}
for prob_id, solver in enumerate(lns_solvers):
    sol = get_best_before(time_cutoff, solver.solutions_cache)
    solutions_cutoff_dict[prob_id] = dict(solution=sol, objective=sol.objective)
    print(f"LNS cutoff Sol: id={prob_id}, obj={sol.objective}, solving_time={'{:.2f}'.format(sol.time_to_find)}")


LNS cutoff Sol: id=0, obj=255, solving_time=4.34
LNS cutoff Sol: id=1, obj=213, solving_time=1.36
LNS cutoff Sol: id=2, obj=258, solving_time=1.26
LNS cutoff Sol: id=3, obj=310, solving_time=8.27
LNS cutoff Sol: id=4, obj=260, solving_time=3.72
LNS cutoff Sol: id=5, obj=291, solving_time=8.15
LNS cutoff Sol: id=6, obj=451, solving_time=6.38
LNS cutoff Sol: id=7, obj=416, solving_time=8.84
LNS cutoff Sol: id=8, obj=470, solving_time=9.31


In [75]:
exact_objs = [194, 164, 199, 215, 189, 201, 193]
exact_times = [23.1, 24.65, 137.52, 365.29, 2678.30, 2127.43, 19502.54]
for prob_id in [0, 1, 2, 3, 4, 5, 6]:
    solver = lns_solvers[prob_id]
    lns_obj = solver.best_cached_solution.objective
    lns_time = solver.best_cached_solution.time_to_find
    lns_timelimit = solver.time_limit
    exact_obj = exact_objs[prob_id]
    exact_time = exact_times[prob_id]
    gap = get_gap(exact_val=exact_obj, heuristic_val=lns_obj) * 100
    speedup_best = (exact_time / lns_time)
    speedup_limit = (exact_time / lns_timelimit)
    print(f"for {prob_id} gap: {'{:.2f}'.format(gap)}, best speedup: {'{:.2e}'.format(speedup_best)}, limit speedup: {'{:.2e}'.format(speedup_limit)} ")
    # print(f"for {prob_id} gap: {gap}, speedup: {'{:.2e}'.format(speedup)}") 

for 0 gap: 23.92, best speedup: 5.33e+00, limit speedup: 4.62e+00 
for 1 gap: 23.00, best speedup: 1.81e+01, limit speedup: 4.93e+00 
for 2 gap: 22.87, best speedup: 1.10e+02, limit speedup: 2.75e+01 
for 3 gap: 29.04, best speedup: 2.06e+01, limit speedup: 1.22e+01 
for 4 gap: 27.31, best speedup: 7.20e+02, limit speedup: 8.93e+01 
for 5 gap: 25.28, best speedup: 1.80e+02, limit speedup: 7.09e+01 
for 6 gap: 44.70, best speedup: 3.27e+02, limit speedup: 3.25e+02 


In [79]:
exact_objs = [194, 164, 199, 215, 189, 201, 193]
exact_times = [23.1, 24.65, 137.52, 365.29, 2678.30, 2127.43, 19502.54]
for prob_id in [6]:
    solver = lns_solvers_2[prob_id - 6]
    lns_obj = solver.best_cached_solution.objective
    lns_time = solver.best_cached_solution.time_to_find
    lns_timelimit = solver.time_limit
    exact_obj = exact_objs[prob_id]
    exact_time = exact_times[prob_id]
    gap = get_gap(exact_val=exact_obj, heuristic_val=lns_obj) * 100
    speedup_best = (exact_time / lns_time)
    speedup_limit = (exact_time / lns_timelimit)
    print(f"for {prob_id} gap: {'{:.2f}'.format(gap)}, best speedup: {'{:.2e}'.format(speedup_best)}, limit speedup: {'{:.2e}'.format(speedup_limit)} ")

for 6 gap: 25.48, best speedup: 5.59e+02, limit speedup: 1.95e+02 


In [1]:
exact_obj

NameError: name 'exact_obj' is not defined